### Load and preprocess the dataset

In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match ResNet input
        transforms.RandomHorizontalFlip(),  # Data augmentation
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Standard ResNet normalization
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load dataset
data_dir = 'dataset'  
datasets = {
    phase: datasets.ImageFolder(os.path.join(data_dir, phase), transform=data_transforms[phase])
    for phase in ['train', 'val']
}

# Create dataloaders
dataloaders = {
    phase: DataLoader(datasets[phase], batch_size=32, shuffle=(phase == 'train'), num_workers=4)
    for phase in ['train', 'val']
}

# Dataset sizes and class names
dataset_sizes = {phase: len(datasets[phase]) for phase in ['train', 'val']}
class_names = datasets['train'].classes
print(f"Classes: {class_names}")

### Load pre trained ResNet50

In [ ]:
import torch
from torchvision import models

# Load pre trained ResNet50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)

# Modify classifier head
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 19)  # 19 bowerbirds
model = model.to(device)


### Define training pipeline

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler
import time

criterion = torch.nn.CrossEntropyLoss() # Loss function
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # Optimizer
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Learning rate scheduler

# Training function
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass + optimize only in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Track statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Actual training of the model
model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)


### Model evaluation: Confusion matrix and classification report

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

val_loader = dataloaders['val']

# Get predictions on the validation set
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=class_names))
print(confusion_matrix(all_labels, all_preds))